In [3]:
import pulp
import pandas as pd
import datetime as dt
pd.options.mode.chained_assignment = None  # Option to suppress the warning

# Import data
food_t = pd.read_csv('Data/Cleaned/Food.csv', parse_dates=['date', 'time'])
items = pd.read_csv('Data/Cleaned/MFP meals scrapped.csv', parse_dates=['date'])

# Keep only dates in both datasets
food_t = food_t[food_t['date'].isin(items['date'])]
items = items[items['date'].isin(food_t['date'])]

# Ensure time is in datetime format
food_t['time'] = pd.to_datetime(food_t['time'])
# Ensure date is in datetime format
items['date'] = pd.to_datetime(items['date'])

# Define preferred times in hours (24-hour format)
preferred_times = {
    'breakfast': 7,  # 7 AM
    'lunch': 13,     # 1 PM
    'dinner': 20     # 8 PM
}

# Get unique dates from data
unique_dates = pd.Series(pd.unique(food_t['date']))

# Iterate through each date
results_df = pd.DataFrame()  # Initialize an empty DataFrame to store all results
for current_date in unique_dates:
    # Filter data for the current date
    daily_food_t = food_t[food_t['date'] == current_date]
    daily_items = items[items['date'] == current_date]

    # Rebase index
    daily_food_t.index = range(len(daily_food_t))
    daily_items.index = range(len(daily_items))

    # This approach is safe as long as `daily_food_t` is a confirmed copy
    daily_food_t['time_hours'] = daily_food_t['time'].dt.hour
  
    # Create the optimization problem
    prob = pulp.LpProblem("FoodScheduling", pulp.LpMinimize)

    # Variables
    x = pulp.LpVariable.dicts("assignment", ((i, j) for i in range(len(daily_items)) for j in range(len(daily_food_t))),
                              cat=pulp.LpBinary)

    # Objective function to minimize the total deviation from preferred meal times
    objective = pulp.lpSum(
        abs(daily_food_t.at[j, 'time_hours'] - preferred_times[daily_items.at[i, 'meal']]) * x[(i, j)]
        for i in range(len(daily_items)) for j in range(len(daily_food_t))
        if daily_items.at[i, 'meal'] in preferred_times
    )
    prob += objective

    # Constraints
    # Each item is assigned to exactly one time slot
    for i in range(len(daily_items)):
        prob += pulp.lpSum(x[(i, j)] for j in range(len(daily_food_t))) == 1, f"UniqueAssignment_{i}"

    # Ensure every time slot has at least one meal assigned
    for j in range(len(daily_food_t)):
        prob += pulp.lpSum(x[(i, j)] for i in range(len(daily_items))) >= 1, f"NonEmptyTimeSlot_{j}"

    # Calorie constraints with tolerance
    tolerance = 2
    for j in range(len(daily_food_t)):
        total_calories = pulp.lpSum(daily_items.at[i, 'calories'] * x[(i, j)] for i in range(len(daily_items)))
        target_calories = daily_food_t.at[j, 'calories']
        # Convert target_calories to integer before using it in arithmetic operations
        target_calories = float(target_calories)
        prob += total_calories >= (target_calories - tolerance), f"MinCaloriesTime_{j}"
        prob += total_calories <= (target_calories + tolerance), f"MaxCaloriesTime_{j}"

    # Solve the problem
    status = prob.solve()
    print(f"Status on {current_date}: {pulp.LpStatus[status]}")

    # Extract and print results for the current date
    daily_results = []
    for i in range(len(daily_items)):
        for j in range(len(daily_food_t)):
            if x[(i, j)].varValue == 1:
                daily_results.append({
                    'date': current_date,
                    'food': daily_items.at[i, 'food'],
                    'meal': daily_items.at[i, 'meal'],
                    'time': daily_food_t.at[j, 'time']
                })

    daily_results_df = pd.DataFrame(daily_results)
    results_df = pd.concat([results_df, daily_results_df], ignore_index=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_32240\2331535203.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  food_t = pd.read_csv('Data/Cleaned/Food.csv', parse_dates=['date', 'time'])


Status on 2024-03-16 00:00:00: Optimal
Status on 2024-03-17 00:00:00: Optimal
Status on 2024-03-18 00:00:00: Optimal
Status on 2024-03-19 00:00:00: Optimal
Status on 2024-03-20 00:00:00: Optimal
Status on 2024-03-21 00:00:00: Infeasible
Status on 2024-03-22 00:00:00: Infeasible
Status on 2024-03-23 00:00:00: Optimal
Status on 2024-03-24 00:00:00: Optimal
Status on 2024-03-25 00:00:00: Optimal
Status on 2024-03-26 00:00:00: Optimal
Status on 2024-03-27 00:00:00: Optimal
Status on 2024-03-28 00:00:00: Optimal
Status on 2024-03-29 00:00:00: Optimal
Status on 2024-03-30 00:00:00: Optimal
Status on 2024-03-31 00:00:00: Infeasible
Status on 2024-04-01 00:00:00: Optimal
Status on 2024-04-02 00:00:00: Optimal
Status on 2024-04-03 00:00:00: Optimal
Status on 2024-04-04 00:00:00: Optimal
Status on 2024-04-05 00:00:00: Optimal
Status on 2024-04-06 00:00:00: Optimal
Status on 2024-04-07 00:00:00: Optimal
Status on 2024-04-08 00:00:00: Optimal
Status on 2024-04-09 00:00:00: Infeasible
Status on 202

In [4]:
# Copy the time onto the items dataset
merged_results = pd.merge(items, results_df, on=['food', 'date', 'meal'], how='left')

# Get only the time from time time column (not the date) and put it as the third column
merged_results['time'] = merged_results['time'].dt.time
merged_results = merged_results[['date', 'meal', 'time', 'food', 'quant', 'calories', 'carbs', 'fat', 'protein', 'sodium', 'sugar']]

merged_results.head()

,date,meal,time,food,quant,calories,carbs,fat,protein,sodium,sugar
0,2024-03-16,breakfast,10:51:00,"Galia - Melon, 80 g - 1/4 melon",80.0,30.0,7.0,0.0,1.0,15.0,7.0
1,2024-03-16,breakfast,10:51:00,"Aquilea - Magnesio + Potasio, 1 Pastilla",1.0,10.0,0.0,0.0,0.0,0.0,0.0
2,2024-03-16,lunch,16:37:00,"Arroz Blanco - Arroz Blanco Cocido, 120 gram",120.0,223.0,48.0,0.0,5.0,629.0,0.0
3,2024-03-16,lunch,16:37:00,"Kimchi, 1 cup",1.0,23.0,4.0,1.0,2.0,747.0,2.0
4,2024-03-16,lunch,16:37:00,"Tofu, 3 ounce",3.0,40.0,1.0,2.0,4.0,2.0,0.0


In [ ]:
# Save onto csv
merged_results.to_csv('Data/Cleaned/MealSchedule.csv', index=False)